In [1]:
from collections import defaultdict
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from consent.consistency.comply_util import site_to_contras, get_comply
from consent.consistency.cookie_pref_match import cookie_pref_match
from consent.consistency.util import get_scan_root_dir, get_scan_dirs # , FIG_DIR uncomment to save fig.
from consent.data.pref_menu_scan.cookie_pref_reader import read_cookie_prefs_in_scans
from consent.data.pref_menu_scan.har_cookie_reader import read_postrej_sent_cookies_in_scans

cur_set = '0k_20k' # '20k_100k'
overwrite = True
SCAN_ROOT_DIR = get_scan_root_dir('sf')
SCAN_DIRS = [SCAN_ROOT_DIR / f'pref_menu_scan_{cur_set}']
# SCAN_DIRS = get_scan_dirs('us')[:]
output_suffix = '_' + cur_set # '0k_20k'  # '20k_100k' #'60k_80k' # '40k_60k' # # '100k_200k'; done: '20k_40k' '0k_20k'
print('Scan root dir:', SCAN_ROOT_DIR)
print('Output suffix:', output_suffix)
print('Scan dirs:', SCAN_DIRS)

Scan root dir: /mnt/sdd/ducbui/projects/data2/consent/2024-10-10
Output suffix: _0k_20k
Scan dirs: [PosixPath('/mnt/sdd/ducbui/projects/data2/consent/2024-10-10/pref_menu_scan_0k_20k')]


In [2]:
# Get cookie prefs = cookie decls + prefs
cookie_prefs_file = SCAN_ROOT_DIR / f'cookie_prefs{output_suffix}.parquet'

if not cookie_prefs_file.exists() or overwrite:
    save_cookie_decls_file = SCAN_ROOT_DIR / f'cookie_decls{output_suffix}.parquet'
    print(f'To save to {save_cookie_decls_file}')
    cookie_prefs = read_cookie_prefs_in_scans(SCAN_DIRS, save_cookie_decls_file)  # took 2.3min for top 50k-site scan
    cookie_prefs.to_parquet(cookie_prefs_file); print(f'Written to {cookie_prefs_file}')
else:
    cookie_prefs = pd.read_parquet(cookie_prefs_file)
cookie_prefs.head()
# TODO: add log_file_checker

To save to /mnt/sdd/ducbui/projects/data2/consent/2024-10-10/cookie_decls_0k_20k.parquet


File iberia.com consent_resources.json has no data
https://cdn.cookielaw.org/consent/03b96273-0ebc-43cb-8885-cdfaeadad5b9/1ce77e97-5389-4791-87b0-e40bddba01f0/en.json WARNING: Cookie list has session cookie but duration > 0
https://cdn.cookielaw.org/consent/c570eb43-1169-4222-914c-38a09bd70ca0/018de4cc-f70c-7263-bc39-1d31f46d243a/en.json WARNING: Cookie list has session cookie but duration > 0
https://cdn-ukwest.onetrust.com/consent/86f42b6d-7a31-469c-804c-319643173d42/018dfc06-450f-70ae-81ef-536da56b2895/en.json WARNING: Cookie list has session cookie but duration > 0
Written to /mnt/sdd/ducbui/projects/data2/consent/2024-10-10/cookie_decls_0k_20k.parquet
Written to /mnt/sdd/ducbui/projects/data2/consent/2024-10-10/cookie_prefs_0k_20k.parquet


,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name,consent
0,.AxcmXFaxAuth,fax.nextiva.com,0 days,C0001,Strictly Necessary Cookies,always active,nextiva.com,onetrust,en.json,True
1,__cf_bm,nextiva.com,0 days,C0001,Strictly Necessary Cookies,always active,nextiva.com,onetrust,en.json,True
2,_ce.cch,nextiva.com,Session,C0001,Strictly Necessary Cookies,always active,nextiva.com,onetrust,en.json,True
3,ao,sso.nextiva.com,0 days,C0001,Strictly Necessary Cookies,always active,nextiva.com,onetrust,en.json,True
4,autolaunch_triggered,sso.nextiva.com,0 days,C0001,Strictly Necessary Cookies,always active,nextiva.com,onetrust,en.json,True


In [3]:
print("Found cookie libraries:")
cookie_prefs[['site', 'lib_name']].drop_duplicates().lib_name.value_counts()

Found cookie libraries:


lib_name
onetrust     1334
cookiebot     110
Name: count, dtype: int64

In [4]:
cookielist_sites = sorted(cookie_prefs.site.unique().tolist())
n_cookielist_sites = len(cookielist_sites)
assert cookie_prefs.site.nunique() == n_cookielist_sites
n=10
print("Sites with cookie prefs:", n_cookielist_sites, f', first {n} sites:' , cookielist_sites[:n])

Sites with cookie prefs: 1444 , first 10 sites: ['247sports.com', '2k.com', '2ksports.com', '9gag.com', 'aber.ac.uk', 'acast.com', 'accedo.tv', 'accenture.com', 'accor.com', 'acehardware.com']


In [5]:
site_to_libname = {row['site']: row['lib_name'] for row in cookie_prefs[['site', 'lib_name']].drop_duplicates().to_dict('records')}

In [6]:
# Quick checking the cookie declaration naming
print(len(cookie_prefs[cookie_prefs.name.str.endswith('#')]) / len(cookie_prefs))
print(len(cookie_prefs[cookie_prefs.name.str.endswith('xxx')]) / len(cookie_prefs))
print(len(cookie_prefs[cookie_prefs.name.str.endswith('XXX')]) / len(cookie_prefs))
cookie_prefs[cookie_prefs.name.str.endswith('#')]

0.01161702141836269


0.028497849501770704
0.002835126248071881


,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name,consent
101,visitor_id#####,go.nextiva.com,364 days,C0004,Targeting Cookies,inactive landingpage,nextiva.com,onetrust,en.json,False
4,_gd#############,www.wework.com,730 days,C0003,Functional Cookies,always active,wework.com,onetrust,en.json,True
8,_gd#############,cdnjs.cloudflare.com,730 days,C0003,Functional Cookies,always active,wework.com,onetrust,en.json,True
46,_gd#############,j.6sc.co,730 days,C0002,Performance Cookies,active,wework.com,onetrust,en.json,False
44,AMP_unsent_#,commercial.cdn.aws.placeit.net,Persistent,statistics,statistics,None,placeit.net,cookiebot,cc.js,False
...,...,...,...,...,...,...,...,...,...,...
140,visitor_id#####,click.outreach.io,3649 days,4,Targeting Cookies,active,outreach.io,onetrust,en.json,False
147,visitor_id#####,pi.pardot.com,3649 days,4,Targeting Cookies,active,outreach.io,onetrust,en.json,False
170,visitor_id#####,pardot.com,3649 days,4,Targeting Cookies,active,outreach.io,onetrust,en.json,False
172,_gd#############,j.6sc.co,729 days,4,Targeting Cookies,active,outreach.io,onetrust,en.json,False


In [7]:
# # Distribution of errors
# print("Libs in failed-to-crawl sites")
# errors.consent_lib_name.value_counts()
# from ooutil.df_util import expand_dict_col
# ot_errs = errors[errors.consent_lib_name == 'onetrust']
# errors_dict = pd.DataFrame(ot_errs.error.tolist()).explode('reason')
# # Uncomment to show
# # errors_dict.reason.value_counts()

# Main analysis

In [8]:
# import sys; import importlib; importlib.reload(sys.modules['consent.data.pref_menu_scan.postrej_cookie_reader'])
from consent.data.pref_menu_scan.har_cookie_reader import read_postrej_sent_cookies_in_scans

cookies_cache_file = SCAN_ROOT_DIR / f'scan{output_suffix}.parquet'  # 'raw_postrej_sent_cookies.parquet'

if not cookies_cache_file.exists() or overwrite:
    sent_cookies = read_postrej_sent_cookies_in_scans(SCAN_DIRS)
    if cookies_cache_file: sent_cookies.to_parquet(cookies_cache_file); print(f"Written to {cookies_cache_file}")
else:
    sent_cookies = pd.read_parquet(cookies_cache_file)

print(f"Number sent cookies read: {len(sent_cookies):,d}")
sent_cookies.head(3)

Post rej cookie not contain consent cookie, skrill.com/postrej_0.har.xz not have cookie OptanonConsent of .skrill.com on path /
Post rej cookie not contain consent cookie, loggly.com/postrej_0.har.xz not have cookie OptanonConsent of .www.loggly.com on path /
Post rej cookie not contain consent cookie, lidl.com/postrej_0.har.xz not have cookie OptanonConsent of .lidl.com on path /
Unmatch url and consent cookie domain: page_url='https://login.microsoftonline.com/44467e6f-462c-4ea2-823f-7800de5434e3/saml2?SAMLRequest=fVLLbtswEPwVgneJNEVbMmE5cGMUNZA0RqTk0EtB0auYAEW6JJW0f1%2F6FSSXnHd2ZmdmFzd%2FB4NewQftbI0nOcUIrHI7bV9q%2FNR%2Bzyp8s1wEORh2EKsx7u0j%2FBkhRJQWbRDnSY1Hb4WTQQdh5QBBRCWa1f2dYDkVB%2B%2BiU85gtAoBfExSt86GcQDfgH%2FVCp4e72q8j%2FEQBCFhr7vOGYj7XBujrdMhh91I9O5AElWvDZDTJeSowMj2oWlJ0zxgtE53aSvjycuVzrgXbfNBK%2B%2BC66OziRJy5QbCOZ%2BVMOszPmMq4yBZVrGiz8qK0h1MecGhICeDGG3WNf5dQiHLvuwkqyTv56rjczph04LOpkUHrEiwEEbY2BCljTVmlPFsQjM6b2klaCkmVV5My18YbS%2BRfNP2HPVX%2BXVnUBA%2F2nabHe1i9HytLAHwpSBxUvcfm%2F

,name,value,domain,path,expires,size,httpOnly,secure,session,sameSite,priority,sameParty,sourceScheme,sourcePort,request_url,site,page_url,partitionKey
0,_ga_914YM665GV,GS1.1.1728455127.1.1.1728455208.51.0.0,.nextiva.com,/,1.763015e+09,52,False,False,False,NaN,Medium,False,Secure,443,https://www.nextiva.com/cdn-cgi/image/width=96...,nextiva.com,https://www.nextiva.com/products/whats-new,NaN
1,__hssrc,1,.nextiva.com,/,-1.000000e+00,8,False,False,True,Lax,Medium,False,Secure,443,https://www.nextiva.com/wp-content/themes/next...,nextiva.com,https://www.nextiva.com/products/whats-new,NaN
2,PHPSESSID,2d6f44931491ff61c8d8ca2db65c316e,www.nextiva.com,/,-1.000000e+00,41,False,False,True,NaN,Medium,False,Secure,443,https://www.nextiva.com/cdn-cgi/image/width=15...,nextiva.com,https://www.nextiva.com/products/whats-new,NaN


In [9]:
# prj = post-rejection
# Old way: consider all flows
# prj_sent_cookies = sent_cookies[['domain', 'expires', 'name', 'path', 'sameSite', 'secure', 'value', 'request_url', 'site']].drop_duplicates()
# prj_br_cookies = prj_sent_cookies[['domain', 'expires', 'name', 'path', 'sameSite', 'secure', 'site']].drop_duplicates()
# New way: consider cookies only (ignore the request url, which is not nused in get_comply cookie matching anyway)
prj_sent_cookies = sent_cookies[['name', 'domain', 'path', 'site']].drop_duplicates()

In [10]:
# print(f"Num captured sent cookies: {len(sent_cookies):,d}")
# print(f"Num unique captured cookies: {len(prj_sent_cookies):,d}")
# n_br_cookies = len(prj_br_cookies)
# print(f"Num unique browser cookies: {n_br_cookies:,d} on {prj_br_cookies.site.nunique():,d} websites") # and {sent_cookies.page_url.nunique():,d} pages")

## Analyze cookies compliance    

In [ ]:
sites = set(cookie_prefs.site)
print(f"Num sent cookies in the cookie prefs sites: {len(prj_sent_cookies[prj_sent_cookies.site.isin(sites)]):,d}")

Num sent cookies in the cookie prefs sites: 58,035


In [12]:
import sys; import importlib; importlib.reload(sys.modules['consent.consistency.comply_util'])
from consent.consistency.comply_util import get_comply

print("faster: for cookie_pref in cookie_pref_set[cookie_pref_set.name == acookie['name']]:")
all_complies = get_comply(cookie_prefs, prj_sent_cookies)
all_complies.head()

faster: for cookie_pref in cookie_pref_set[cookie_pref_set.name == acookie['name']]:


  0%|          | 3/1444 [00:00<01:11, 20.20it/s]

 89%|████████▉ | 1283/1444 [04:17<00:08, 19.76it/s]

 18%|█▊        | 262/1444 [00:50<02:09,  9.10it/s]

Error fuzzy name match pref_name='mf_[0cc3b612-a97d-4bd0-b496-3ed0686d4603]' cookie_name='_gcl_au' bad character range d-4 at position 20
Error fuzzy name match pref_name='mf_[0cc3b612-a97d-4bd0-b496-3ed0686d4603]' cookie_name='AWSALBTG' bad character range d-4 at position 20
Error fuzzy name match pref_name='mf_[0cc3b612-a97d-4bd0-b496-3ed0686d4603]' cookie_name='_ga' bad character range d-4 at position 20
Error fuzzy name match pref_name='mf_[0cc3b612-a97d-4bd0-b496-3ed0686d4603]' cookie_name='__eoi' bad character range d-4 at position 20
Error fuzzy name match pref_name='mf_[0cc3b612-a97d-4bd0-b496-3ed0686d4603]' cookie_name='optimizelyEndUserId' bad character range d-4 at position 20
Error fuzzy name match pref_name='mf_[0cc3b612-a97d-4bd0-b496-3ed0686d4603]' cookie_name='_gid' bad character range d-4 at position 20
Error fuzzy name match pref_name='mf_[0cc3b612-a97d-4bd0-b496-3ed0686d4603]' cookie_name='__gpi' bad character range d-4 at position 20
Error fuzzy name match pref_name

100%|██████████| 1444/1444 [04:41<00:00,  5.14it/s]


,name,domain,path,site,comply
0,_ga_914YM665GV,.nextiva.com,/,nextiva.com,incorrect
1,__hssrc,.nextiva.com,/,nextiva.com,omit
2,PHPSESSID,www.nextiva.com,/,nextiva.com,comply
3,_ga,.nextiva.com,/,nextiva.com,incorrect
4,OptanonConsent,.nextiva.com,/,nextiva.com,comply


In [ ]:
# Analyze complies here
complies = all_complies[['name', 'domain', 'path', 'site', 'comply']].drop_duplicates()
comply_sites_data = defaultdict(list)
for comply_type, comply_group in complies.groupby('comply'):
    n_sites = comply_group.site.nunique()
    comply_sites_data['comply_type'].append(comply_type)
    comply_sites_data['num_sites'].append(n_sites)
    comply_sites = pd.DataFrame(comply_sites_data).sort_values(by=['num_sites'], ascending=False)

nsites = all_complies.site.nunique()
comply_counts = complies.comply.value_counts()
comply_sites['num_sites_percent'] = comply_sites['num_sites'] / nsites * 100
comply_sites['num_cookies'] = comply_sites['comply_type'].map(comply_counts)
comply_sites['num_cookies_percent'] = comply_sites['num_cookies'] / comply_sites['num_cookies'].sum() * 100  # need to read scan_*.parquet for n_br_cookies, but may be unnecessary
comply_sites['num_cookies_per_site'] = comply_sites['num_cookies'] / comply_sites['num_sites']

# Do not count comply/correct enforcement: which require detecting all possible
comply_sites = comply_sites[comply_sites.comply_type != 'comply']
comply_sites

,comply_type,num_sites,num_sites_percent,num_cookies,num_cookies_percent,num_cookies_per_site
3,omit,1346,93.537179,29589,50.984751,21.982912
2,incorrect,1233,85.684503,18889,32.547601,15.319546
0,ambiguous,51,3.544128,79,0.136125,1.549020


In [ ]:
# Record the analysis results
all_complies_file = SCAN_ROOT_DIR / f'all_complies{output_suffix}.parquet'
all_complies.to_parquet(all_complies_file); print(f"Written {len(all_complies):,d} records of all_complies to {all_complies_file}")

Written 58,035 records of all_complies to /mnt/sdd/ducbui/projects/data2/consent/2024-10-10/all_complies_0k_20k.parquet


In [15]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

raise StopExecution
import sys; sys.exit(0)

In [16]:
complies.comply

0              omit
1              omit
2              omit
3              omit
4              omit
            ...    
115682    incorrect
115683    incorrect
115684         omit
115685       comply
115686       comply
Name: comply, Length: 115687, dtype: object

In [17]:
nsites = all_complies.site.nunique(); nsites
# all_complies.site.unique()

3412

In [18]:
detected_contra_sites = [s for s, contras in site_to_contras.items() if len(contras) > 0]; len(detected_contra_sites)

0

In [19]:
complies = all_complies[ ['name', 'domain', 'site', 'comply',]].drop_duplicates()
comply_counts = complies.comply.value_counts()
comply_counts

omit         60214
incorrect    35575
comply       19067
ambiguous      271
Name: comply, dtype: int64

In [20]:
complies_sites = set(all_complies.site)
sent_cookies_com = sent_cookies[sent_cookies.site.isin(complies_sites)]
prj_sent_cookies_com = sent_cookies_com[['domain', 'expires', 'name', 'path', 'sameSite', 'secure', 'value', 'request_url', 'site']].drop_duplicates()
prj_br_cookies_com = prj_sent_cookies_com[['domain', 'expires', 'name', 'path', 'sameSite', 'secure', 'site']].drop_duplicates()
print(f"Num captured sent cookies: {len(sent_cookies_com):,d}")
print(f"Num unique captured cookies: {len(prj_sent_cookies_com):,d}")

n_br_cookies_com = len(prj_br_cookies_com)
print(f"Num unique browser cookies: {n_br_cookies_com:,d} on {prj_br_cookies_com.site.nunique():,d} websites") # and {sent_cookies.page_url.nunique():,d} pages")

Num captured sent cookies: 12,865,663
Num unique captured cookies: 6,925,444
Num unique browser cookies: 276,967 on 3,412 websites


# Analyze contras

In [21]:
# Way 1: compute contra sites by dynamic analysis: this should be lower than statically analyzing prefs
# because we cannot check all combinations of consent modes.
from consent.consistency.comply_util import get_appr_rej_sets
contra_data = []
for asite in cookie_prefs.site.unique():
    site_prefs = cookie_prefs[cookie_prefs.site == asite]
    _, _, contras = get_appr_rej_sets(site_prefs)
    for contra in contras:
        contra['site'] = asite
        contra_data.append(contra)
contra_sites = pd.DataFrame(contra_data)
contra_sites.head()

,domain,name,site
0,www.libertylondon.com,dwanonymous_xxxxxxxx,libertylondon.com
1,script.hotjar.com,_hjIncludedInPageviewSample,tartecosmetics.com
2,tartecosmetics.com,_hjIncludedInPageviewSample,tartecosmetics.com
3,tartecosmetics.com,_hjFirstSeen,tartecosmetics.com
4,script.hotjar.com,_hjIncludedInSessionSample,tartecosmetics.com


In [22]:
assert set(detected_contra_sites) == set(contra_sites.site), "this is the same method with above, so the result"

AssertionError: this is the same method with above, so the result

In [ ]:
set(detected_contra_sites) - set(contra_sites.site)
set(contra_sites.site) - set(detected_contra_sites)
# This may be the sites excluded due to testing cookies in the preferences.

{'247sports.com',
 'abbvie.com',
 'acquia.com',
 'adobe.io',
 'adweek.com',
 'aljazeera.com',
 'allaboutvision.com',
 'allegiantair.com',
 'allergan.com',
 'alteryx.com',
 'americanbar.org',
 'aon.com',
 'ariel.co.uk',
 'asos.com',
 'autosport.com',
 'avalara.com',
 'avast.com',
 'avastbrowser.com',
 'barnesandnoble.com',
 'bazaarvoice.com',
 'bbva.com',
 'bd.com',
 'behance.net',
 'belkin.com',
 'betsson.com',
 'bitdefender.com',
 'bitdefender.net',
 'bkstr.com',
 'blackboard.com',
 'bplans.com',
 'brightside.me',
 'bromium-online.com',
 'callofduty.com',
 'cambridge.org',
 'carbonite.com',
 'cheapoair.com',
 'chrysler.com',
 'cisco.com',
 'citationmachine.net',
 'clickmeeting.com',
 'clicktale.net',
 'cloudbees.com',
 'cnet.com',
 'colt.net',
 'commonsensemedia.org',
 'contentsquare.com',
 'conviva.com',
 'creighton.edu',
 'curalate.com',
 'dc.com',
 'dccomics.com',
 'discogs.com',
 'dynamicyield.com',
 'easybib.com',
 'eidos.com',
 'elderscrollsonline.com',
 'evri.com',
 'ew.com',
 

In [ ]:
# Way 2: compute contra sites by DataFrame group by
contra_cookies_dfs = []
for _, same_cookies in cookie_prefs.groupby(['name', 'domain', 'site']):
    consent_modes = same_cookies.consent_mode.unique()
    if len(consent_modes) >= 2 and same_cookies.category.nunique() > 1: # and 'always active' in consent_modes:
        contra_cookies_dfs.append(same_cookies)
contra_cookies = pd.concat(contra_cookies_dfs).drop_duplicates()

In [ ]:
print("Number sites with contra cookies:", contra_cookies.site.nunique())

Number sites with contra cookies: 184


In [ ]:
# set(contra_cookies.site) - set(contra_sites.site)

In [ ]:
print("Num contra cookie settings (double categories):", len(contra_cookies), f"settings  on {contra_cookies.site.nunique()} sites")
print("Contras with 1 always-active:", contra_sites.site.nunique())
print("Contras with both editable:", len(set(contra_cookies.site) - set(contra_sites.site)))

Num contra cookie settings (double categories): 648 settings  on 184 sites
Contras with 1 always-active: 192
Contras with both editable: 0


In [ ]:
contra_cookies[contra_cookies.site == 'elsevier.nl']

,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name,consent


In [ ]:
# assert len(detected_contra_sites) <= contra_sites.site.nunique() < len(detected_contra_sites) + 2

In [ ]:
complies[complies.comply == 'ambiguous']

,name,domain,site,comply
173,TS013559a7,www.mastercard.us,mastercard.com,ambiguous
1236,_qubitTracker,.www.very.co.uk,very.co.uk,ambiguous
2365,G_ENABLED_IDPS,.www.glassdoor.com,glassdoor.com,ambiguous
2372,trs,www.glassdoor.com,glassdoor.com,ambiguous
2375,_gid,.glassdoor.com,glassdoor.com,ambiguous
...,...,...,...,...
59468,ak_bmsc,.mulesoft.com,mulesoft.com,ambiguous
59481,JSESSIONID,training.mulesoft.com,mulesoft.com,ambiguous
60397,TLTSID,.bkstr.com,bkstr.com,ambiguous
60412,dtSa,.bkstr.com,bkstr.com,ambiguous


In [ ]:
cookie_prefs[ (cookie_prefs.site == 'michaels.com') & (cookie_prefs.name == 'utag_main') ]

,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name,consent
88,utag_main,.michaels.com,365 days,C0001,Strictly Necessary Cookies,always active,michaels.com,onetrust,en.json,True
148,utag_main,michaels.com,365 days,C0002,Performance Cookies,active,michaels.com,onetrust,en.json,False


In [ ]:
cookie_prefs[ (cookie_prefs.site == 'statista.com') & (cookie_prefs.name == 'id') ]

,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name,consent
74,id,m6r.eu,90 days,C0001,Necessary Cookies,always active,statista.com,onetrust,en.json,True
131,id,m6r.eu,0 days,C0004,Targeting Cookies,inactive,statista.com,onetrust,en.json,False


In [ ]:
# contra_cookies.name.value_counts()[:10]
# contra_cookies[contra_cookies.name == '__cf_bm'].reset_index(drop=True) # .site.nunique()

# Analyze ambiguous flows

In [ ]:
ambi_cookies = complies[complies.comply == 'ambiguous']
print("Num ambiguous cookies:", len(ambi_cookies), "on num sites:", ambi_cookies.site.nunique())
assert len(ambi_cookies) == len(ambi_cookies.drop_duplicates()), "there is some duplicate cookies"


## Analyze no-way-to-opt-out

In [ ]:
# Way 1: compute contra sites by DataFrame group by
nwtoo_cookies_dfs = []
for _, site_cookies in cookie_prefs.groupby(['site']):
    consent_modes = site_cookies.consent_mode.unique().tolist()
    if len(consent_modes) == 1 and consent_modes[0] == 'always active':
        nwtoo_cookies_dfs.append(site_cookies)
nwtoo_cookies = pd.concat(nwtoo_cookies_dfs).drop_duplicates()
nwtoo_cookies.head()

In [ ]:
nwtoo_cookies.site.nunique()

# Analyze inconsistent flows/cookies

In [ ]:
def plot_inconsis_types():
    sns.set(rc={'figure.figsize':(9,9),"font.size":30,"axes.titlesize":30,"axes.labelsize":30},style="white", font_scale = 2.5)

    ax = sns.barplot(x='comply_type', y='num_sites_percent', data=noncomply_sites)
    ax.tick_params(left=True, bottom=True, direction="in")
    ax.set_ylabel('# Websites (%)')
    ax.set_xlabel('Noncompliance Types')
    xticks = [x.get_text().title() for x in ax.get_xticklabels()]
    ax.set(xticklabels=xticks)

    out_file = FIG_DIR / 'noncomply_types_per_site.png'
    # plt.savefig(out_file, dpi=200, bbox_inches='tight'); print(f"Written to {out_file}")

In [ ]:
## Plot distribution of omitted cookie declarations

In [ ]:
omit_complies = complies[(complies.comply == 'omit')] #  & ~complies.name.isin(['OptanonAlertBoxClosed', 'OptanonConsent']) ]
omit_complies.site.nunique()

In [ ]:
omit_complies.site.value_counts()[:7]

In [ ]:
omit_cookies = omit_complies.name.value_counts()[:5].to_frame().reset_index().rename(columns={'index': 'cookie_name'})
omit_cookies['num_sites'] = omit_cookies['cookie_name'].map(lambda cookie_name: omit_complies[omit_complies.name == cookie_name].site.nunique())
print(omit_cookies.head(5)[['cookie_name', 'num_sites']].rename(columns={'cookie_name': 'Cookie Name', 'num_sites': '# Websites'}).to_latex(index=False))

In [ ]:
# set(cookielist_sites) - set(omit_complies.site.unique())
omit_counts = omit_complies.site.value_counts().to_frame().rename(columns={'site': 'num_omit_cookies'})
print("Num omitted sites:", omit_complies.site.nunique())
omit_counts.num_omit_cookies.mean(), omit_counts.num_omit_cookies.std(), omit_counts.num_omit_cookies.min(), omit_counts.num_omit_cookies.max()

In [ ]:
sns.set(rc={'figure.figsize':(12,8),"font.size":35,"axes.titlesize":30,"axes.labelsize":50},style="white", font_scale = 4.5)
sns.set_style("ticks", {"xtick.direction": "in", "ytick.direction": "in"})
plt.rcParams['xtick.major.size'] = 15
plt.rcParams['ytick.major.size'] = 15

ax = sns.ecdfplot(data=omit_counts, x=omit_counts.num_omit_cookies, linewidth=8)
ax.set(xlim=(0, 350))

plt.xlabel('# Omitted Cookies/Website')
plt.ylabel('CDF')
# out_file = FIG_DIR / 'eu_omit_cookie_cdf.png'
# plt.savefig(out_file, dpi=200, bbox_inches='tight'); print(f"Written to {out_file}")

In [ ]:
plot_df = omit_counts[:50]

sns.set(rc={'figure.figsize':(24,8),"font.size":35,"axes.titlesize":35,"axes.labelsize":25},style="white", font_scale=1.5)
ax = sns.barplot(x=plot_df.index, y=plot_df.num_omit_cookies)
ticks = plt.xticks(rotation=45, ha='right')

ax.tick_params(left=True, bottom=False, direction="in")
ax.set_ylabel('# Omitted Cookies')
ax.set_xlabel('Website')

# out_file = FIG_DIR / 'omit_site.png'
# plt.savefig(out_file, dpi=200, bbox_inches='tight'); print(f"Written to {out_file}")

In [ ]:
def plot_histogram_omit():
    sns.set(rc={'figure.figsize':(9,6),"font.size":25,"axes.titlesize":25,"axes.labelsize":15},style="white", font_scale = 1)
    omit_counts_histo = omit_complies.site.value_counts().value_counts()
    sns.barplot(x=omit_counts_histo.index, y=omit_counts_histo.values)
    ticks = plt.xticks(rotation=45, ha='right')

## Plot distribution of incorrectly enforced cookies

In [ ]:
incor_complies = complies[complies.comply == 'incorrect']
# set(cookielist_sites) - set(omit_complies.site.unique())
incor_counts = incor_complies.site.value_counts().iloc[:50]
incor_counts = incor_counts.to_frame().reset_index().sort_values(by=['site', 'index'], ascending=[False, True])
incor_counts.columns

In [ ]:
sns.set(rc={'figure.figsize':(24,8),"font.size":35,"axes.titlesize":35,"axes.labelsize":25},style="white", font_scale=1.5)
ax = sns.barplot(x='index', y='site', data=incor_counts)

ticks = plt.xticks(rotation=45, ha='right')

ax.tick_params(left=True, bottom=True, direction="in")
ax.set_ylabel('# Incorrectly Enforced Cookies')
ax.set_xlabel('Website')

# out_file = FIG_DIR / 'incor_site.png'
# plt.savefig(out_file, dpi=200, bbox_inches='tight'); print(f"Written to {out_file}")

In [ ]:
# len(incor_counts)

In [ ]:
incor_counts[incor_counts.site == 33].reset_index(drop=True)

### Top incorrect cookies

In [ ]:
incor_cookies = complies[complies.comply == 'incorrect']

In [ ]:
# site = 'onetrust.com'
# complies[complies.site == site].reset_index(drop=True)
# cookie_prefs[(cookie_prefs.site == site) & (cookie_prefs.name == '_gid')]

In [ ]:
top_incor_cookies = incor_cookies.name.value_counts().sort_values(ascending=False).to_frame().reset_index().rename(columns={'index':'cookie_name', 'name': 'num_occur'})

In [ ]:
top_incor_cookies['num_sites'] = top_incor_cookies.cookie_name.map(lambda cookie_name: incor_cookies[incor_cookies.name == cookie_name].site.nunique())

In [ ]:
print(top_incor_cookies.head(5)[['cookie_name', 'num_sites']].rename(columns={'cookie_name': 'Cookie Name', 'num_sites': '# Websites'}).to_latex(index=False))

# Narrow cookie consent preferences

In [ ]:
consent_cookies = sent_cookies[sent_cookies.name.isin(['OptanonConsent', 'CookieConsent'])][['domain', 'name', 'site']].drop_duplicates()
narrow_consent_cookies = consent_cookies[~consent_cookies.domain.str.startswith('.')].reset_index(drop=True)

In [ ]:
set(contra_cookies.site).intersection(set(narrow_consent_cookies.site))

In [ ]:
narrow_consent_cookies

# Case studies

In [ ]:
incor_df = complies[complies.comply == 'incorrect']
incor_df.site.value_counts().sort_values(ascending=False).head()

In [ ]:
case_site = 'blabbermouth.net'
incor_df[incor_df.site == case_site]

In [ ]:
# cookie_prefs[(cookie_prefs.site == 'scientificamerican.com') & (cookie_prefs.name == 'MXP_TRACKINGID')]
cookie_prefs[(cookie_prefs.site == case_site) & (cookie_prefs.name == 'IDE')]

In [ ]:
# sent_cookies[(sent_cookies.site == 'scientificamerican.com') & (sent_cookies.name == 'MXP_TRACKINGID')]

In [ ]:
omit_df = complies[complies.comply == 'omit']
omit_df.site.value_counts().sort_values(ascending=False).iloc[20:30]

In [ ]:
print(omit_df[omit_df.site == 'scientificamerican.com'].reset_index().name.tolist())

In [ ]:
'uids' in omit_df[omit_df.site == 'scientificamerican.com'].reset_index().name.tolist()

In [ ]:
omit_df[(omit_df.site == 'scientificamerican.com') & (omit_df.name == 'cftoken')]

In [ ]:
cookie_prefs[(cookie_prefs.site == 'scientificamerican.com') & (cookie_prefs.name == 'ADGRX_UID')]

In [ ]:
all_sites = set(complies.site)
nothing_comply_sites = all_sites - set(complies[complies.comply == 'comply'].site)
print("Sites with no compliant cookies:", nothing_comply_sites)

In [ ]:
for site in ['liveperson.com']: # nothing_comply_sites:
    print(site)
site_cookies = postrej_br_cookies[postrej_br_cookies.site == site]

In [ ]:
acookie = site_cookies[site_cookies.name == 'OptanonAlertBoxClosed'].iloc[0]

In [ ]:
site_prefs = cookie_prefs[cookie_prefs.site == site]
consent_cookies = site_prefs[site_prefs.consent]
consent_cookie_names = consent_cookies.name
site_cookies[site_cookies.name.isin(consent_cookie_names)]
# cookie_name = 'OptanonAlertBoxClosed'
# cookie_pref = site_prefs[site_prefs.name == cookie_name].iloc[0]
# cookie_pref

In [ ]:
consent_cookies

In [ ]:
import sys; import importlib; importlib.reload(sys.modules['consent.consistency.cookie_pref_match'])
from consent.consistency.cookie_pref_match import cookie_pref_match
cookie_pref_match(acookie, cookie_pref, site, verbose=2)

In [ ]:
# Analyze Contras